In [38]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import ManagedOnlineDeployment, Environment, CodeConfiguration
import os


ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="ddfbea37-0529-4562-ac51-2ebbbe770146",
    resource_group_name="cloudServicesS242",
    workspace_name="myWorkspace"
)

In [43]:
code_path = os.path.abspath("./")
# Define your deployment
deployment = ManagedOnlineDeployment(
    name="mnist-deployment-v1",
    endpoint_name="myworkspace-ville",
    model="my_tensorflow_model:3",
    #environment="tensorflow-env:1",
    code_configuration=CodeConfiguration(
        code="./", scoring_script="score.py"
        ),
    environment=Environment(
            conda_file="conda.yml",
            image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
            ),
    
    instance_type="Standard_DS3_v2",
    instance_count=1
)

In [47]:
# Deploy locally
deployment_poller = ml_client.online_deployments.begin_create_or_update(deployment, local=True)


Updating local deployment (myworkspace-ville / mnist-deployment-v1) ..
Building Docker image from Dockerfile..
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04
 ---> 005da6fe2f84
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 6b7850a0392c
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> 098941cf465c
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 3e7fe10c7a38
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> 030f276512d6
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> f25c50624d75
Successfully built f25c50624d75
Successfully tagged myworkspace-ville:mnist-deployment-v1

Starting up endpoint.....Done (0m 45s)


In [4]:
deployment_result = deployment_poller.result()

AttributeError: 'ManagedOnlineDeployment' object has no attribute 'result'

In [5]:
# Get the scoring URI
endpoint_details = ml_client.online_endpoints.get("myworkspace-ville")
print(f"Scoring URI: {endpoint_details.scoring_uri}")

Scoring URI: https://myworkspace-ville.northeurope.inference.ml.azure.com/score
